In [1]:
import matplotlib.pyplot as plt

In [3]:
from collections import Counter
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
#from model import 
from flask import Flask, request, render_template, url_for
from pymongo import MongoClient
from bson import json_util

import requests

In [16]:
client = MongoClient()
db = client.yelp
bdb = db.businesses
rdb = db.reviews

In [3]:
state_summary_data = client.yelp.businesses.aggregate([{'$group':{'_id':'$state', 'rating': {'$avg': '$rating'}, 'count': {'$sum':1} } }, {'$sort': { 'count': -1} }, {'$match': {'count': {'$gte': 100} } } ] )

In [ ]:
city_summary_data = bdb.aggregate([{'$group':{'_id':'$state', 'rating': {'$avg': '$rating'}, 'count': {'$sum':1} } }, {'$sort': { 'count': -1} }, {'$match': {'count': {'$gte': 100} } } ] )
    

In [40]:
city_list = bdb.aggregate([{'$group': {'_id': {'city': '$city', 'state': '$state'}, 'count': {'$sum':1} } }])


In [ ]:
# city_list for 

In [74]:
reviews = rdb.find({'cluster': {'$exists': True}}, {'_id': False, 'bus_id': True})

In [14]:
def find_business_data(kwargs):
    print(kwargs)
    return bdb.find(kwargs, {'_id': False, 'id': True, 'cluster': True, 'sentiment': True})

In [34]:
def business_search_params(state='AZ', city='Phoenix', prob=None, lat=None, long=None):
    bus = {}
    if lat!=None and long!=None:
        bus['$and'] = [{ 'latitude': { '$lt': (lat+0.05)}}, 
                        { 'latitude': { '$gt': (lat-0.05)}},
                        { 'longitude': { '$lt': (long+0.05) }}, 
                        { 'longitude': { '$gt': (long-0.05) }}]
    else: 
        bus['state'] = state
        if city != 'None':
            bus['city'] = city
    
    print(bus['$and'])
    if prob!='None':
        prob=float(prob)
        if '$and' in bus.keys():
            bus['$and'].append({'prob': { '$lt': (prob+0.40)}})
            bus['$and'].append({ 'prob': { '$gt': (prob-0.40)}})
        else:
            bus['$and'] = [{ 'prob': { '$lt': (prob+0.40)}}, 
                            { 'prob': { '$gt': (prob-0.40)}}]
    
    print(bus['$and'])
    return find_business_data(bus)

In [ ]:
business_search_params()

In [76]:
bus_stats = set()

for review in reviews:
    bus_stats.add(review['bus_id'])

In [77]:
len(bus_stats)

1567

In [88]:
city_summary_data = bdb.find({}, {'_id': False, 'id': True, 'state': True, 'city': True, 'rating': True, } )

In [89]:
city_list = [[bus['city'], bus['state'], bus['rating']] for bus in city_summary_data if bus['id'] in bus_stats]

In [90]:
city_df = pd.DataFrame(city_list, columns=['city', 'state', 'rating'])

In [93]:
city_df.groupby(['city', 'state']).agg(['mean', 'count'])

rating      
                                mean count
city                 state                
Anthem               AZ     3.300000     5
Apache Junction      AZ     3.176471    17
Arizona City         AZ     3.000000     1
Avondale             AZ     3.055556     9
Black Canyon City    AZ     3.833333     3
Buckeye              AZ     2.961538    13
Casa Grande          AZ     3.521739    23
Cave Creek           AZ     3.500000     8
Central City         AZ     3.500000     1
Central City Village AZ     3.000000     1
Chandler             AZ     3.380597    67
Coolidge             AZ     5.000000     1
Eloy                 AZ     4.000000     3
Florence             AZ     3.250000     4
Fort McDowell        AZ     3.500000     1
Fort Mcdowell        AZ     3.500000     1
Fountain Hills       AZ     3.454545    11
Gilbert              AZ     3.476190   105
Glendale             AZ     3.386861   137
Glendale Az          AZ     3.500000     1
Goodyear             AZ     3.263158    19
Laveen               AZ     2.277778     9
Maricopa             AZ     3.025000    20
Mesa                 AZ     3.374046   131
Mesa AZ              AZ     3.000000     1
Morristown           AZ     4.500000     2
New River            AZ     4.000000     1
PHOENIX              AZ     4.500000     1
Paradise Valley      AZ     4.100000     5
Peoria               AZ     3.450000    70
Phoenix              AZ     3.475194   645
Pinal County         AZ     5.000000     1
QUEEN CREEK          AZ     2.500000     1
Queen Creek          AZ     3.178571    28
Roosevelt            AZ     4.500000     1
San Tan Valley       AZ     3.800000     5
Scottsdale           AZ     3.717647   170
Somerton             AZ     4.000000     1
Sun City             AZ     3.550000    10
Sun Lakes            AZ     3.100000     5
Superior             AZ     5.000000     1
Surprise             AZ     3.066667    15
Tolleson             AZ     3.571429     7
Tonto Basin          AZ     3.500000     1
Waddell              AZ     4.750000     2
Wickenburg           AZ     2.500000     1
Wittmann             AZ     3.500000     1
Youngtown            AZ     4.000000     1

In [96]:
city = int('10')
city = [c for idx, c in zip(np.arange(0,len(city_list),1), city_list) if idx==city]
print(city)

[['Glendale', 'AZ', 2.5]]


In [23]:
for city in city_list:
    city.keys()

In [24]:
city

{'_id': {'city': 'Bellrose', 'state': 'NY'}, 'count': 1}

In [26]:
city['_id']['city']

'Bellrose'

In [27]:
city['_id']['state']

'NY'

In [32]:
city['count']

1

In [41]:
city_count = [[city['_id']['city'], city['_id']['state'], city['count']] for city in city_list ]

In [42]:
len(city_count)

1702

In [45]:
city_count = np.array(city_count)

In [47]:
city_count[city_count[:,1]=='MD']

array([['Park Hall', 'MD', '2'],
       ['Tilghman Island', 'MD', '9'],
       ['Tilghman', 'MD', '12'],
       ['Great Mills', 'MD', '10'],
       ['Callaway', 'MD', '11'],
       ['Knoxville', 'MD', '8'],
       ['Thurmont', 'MD', '8'],
       ['New Windsor', 'MD', '14'],
       ['Upperco', 'MD', '4'],
       ['Bel Aire', 'MD', '6'],
       ['Bel-Air', 'MD', '6'],
       ['Stevensville', 'MD', '39'],
       ['Bushwood', 'MD', '6'],
       ['Sparks', 'MD', '15'],
       ['Bel Air', 'MD', '523'],
       ['Joppa', 'MD', '69'],
       ['Patuxent River', 'MD', '18'],
       ['Lexington Park', 'MD', '301'],
       ['California', 'MD', '278'],
       ['Myersville', 'MD', '30'],
       ['Baldwin', 'MD', '15'],
       ['Bowleys Quarters', 'MD', '15'],
       ['Clements', 'MD', '9'],
       ['Union Bridge', 'MD', '14'],
       ['Kingsville', 'MD', '39'],
       ['essex', 'MD', '6'],
       ['Point of Rocks', 'MD', '18'],
       ['Westminster', 'MD', '216'],
       ['Reisterstown MD', 'MD', '6'

In [54]:
zip(np.arange(0,len(city_count),1), city_count[:,1])

In [53]:
np.arange(0,len(city_count),1)

array([   0,    1,    2, ..., 1699, 1700, 1701])

In [64]:
np.vstack([np.arange(0,len(city_count),1), city_count[:,0],]).T

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [69]:
cities = set(city_count[:,0])

In [70]:
cities = sorted(cities)

In [73]:
zip(np.arange(0,len(city_count),1), cities)

AttributeError: 'builtin_function_or_method' object has no attribute 'values'

In [97]:
bus_detail = client.yelp.businesses.find({'id': 'zwmps5SXn30g-f5wqg_r9A'})

In [99]:
for x in bus_detail: 
    print(x)

{'_id': ObjectId('5b7f46fc3d38dc7b57a2483e'), 'id': 'zwmps5SXn30g-f5wqg_r9A', 'name': "Garcia's Mexican Restaurants", 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/E5n-7bJ9i2pzIlZk3QDByA/o.jpg', 'url': 'https://www.yelp.com/biz/garcias-mexican-restaurants-phoenix-3?adjust_creative=ihkhZPaVUfUYeK7PoCZp6w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ihkhZPaVUfUYeK7PoCZp6w', 'review_count': 124, 'categories_alias': ['mexican'], 'categories_title': ['Mexican'], 'rating': 3.0, 'latitude': 33.58138, 'longitude': -112.13187, 'transactions': ['pickup'], 'price': '$$', 'address1': '3301 W Peoria Ave', 'address2': '', 'address3': '', 'city': 'Phoenix', 'state': 'AZ', 'zip_code': '85029', 'country': 'US', 'prob': 0.4472792502411728}


In [ ]:
rdb.find({'bus_id': ''}, {'_id': False, 'bus_id': True, 'cluster': True, 'sentiment': True })

In [35]:
business_id = 'oCwJdiYhZS8VM5_LgmQP0g'
business_data = client.yelp.businesses.find({'id': business_id})
businesses = business_search_params(lat = business_data[0]['latitude'], long = business_data[0]['longitude'], prob=business_data[0]['prob'])

bus_search_ids = [bus['id'] for bus in businesses]
print(len(bus_search_ids), 'Businesses Queried')    

[{'latitude': {'$lt': 33.348018999999994}}, {'latitude': {'$gt': 33.248019}}, {'longitude': {'$lt': -111.69437}}, {'longitude': {'$gt': -111.79437}}]
[{'latitude': {'$lt': 33.348018999999994}}, {'latitude': {'$gt': 33.248019}}, {'longitude': {'$lt': -111.69437}}, {'longitude': {'$gt': -111.79437}}, {'prob': {'$lt': 0.8820035580120926}}, {'prob': {'$gt': 0.08200355801209258}}]
{'$and': [{'latitude': {'$lt': 33.348018999999994}}, {'latitude': {'$gt': 33.248019}}, {'longitude': {'$lt': -111.69437}}, {'longitude': {'$gt': -111.79437}}, {'prob': {'$lt': 0.8820035580120926}}, {'prob': {'$gt': 0.08200355801209258}}]}
72 Businesses Queried


In [7]:
for x in business_data:
    print(x)

{'_id': ObjectId('5b7f46fc3d38dc7b57a23029'), 'id': 'oCwJdiYhZS8VM5_LgmQP0g', 'name': 'Arriba Mexican Grill', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/3CtXNOxntHcc9FQncVJeOw/o.jpg', 'url': 'https://www.yelp.com/biz/arriba-mexican-grill-gilbert?adjust_creative=ihkhZPaVUfUYeK7PoCZp6w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ihkhZPaVUfUYeK7PoCZp6w', 'review_count': 207, 'categories_alias': ['mexican'], 'categories_title': ['Mexican'], 'rating': 2.5, 'latitude': 33.298019, 'longitude': -111.74437, 'transactions': [], 'price': '$$', 'address1': '2855 S Market St', 'address2': '', 'address3': '', 'city': 'Gilbert', 'state': 'AZ', 'zip_code': '85295', 'country': 'US', 'prob': 0.4820035580120926}


In [12]:
business_data[0]['latitude']

33.298019